# Fast AI Image Classification: Forest vs Bird

This notebook demonstrates **transfer learning** using Fast AI to classify images into two categories:
- **Forest** 🌲
- **Bird** 🐦

## What is Transfer Learning?
Instead of training a model from scratch, we use a pre-trained model (ResNet18) that already knows how to recognize image features. We then fine-tune it for our specific task.

---

## Step 1: Install Required Libraries

We need:
- `fastai` - The deep learning library
- `duckduckgo_search` - To download images from DuckDuckGo

In [ ]:
# Install libraries (run this once)
!pip install -Uqq fastai duckduckgo_search

## Step 2: Import Libraries and Setup

Import everything we need and create a function to search for images.

In [ ]:
from fastai.vision.all import *
from duckduckgo_search import DDGS

def search_images(term, max_images=30):
    """Search for images using DuckDuckGo and return URLs"""
    print(f"Searching for '{term}'...")
    with DDGS() as ddgs:
        results = ddgs.images(keywords=term, max_results=max_images)
        return L([r['image'] for r in results])

print("✓ Libraries imported successfully!")

## Step 3: Download Training Images

We'll download 30 images each of forests and birds to train our model.

In [ ]:
# Define our categories
searches = 'forest', 'bird'
path = Path('bird_or_not')

# Download and organize images
for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    
    # Download images
    download_images(dest, urls=search_images(f'{o} photo'))
    
    # Resize to manageable size
    resize_images(path/o, max_size=400, dest=path/o)

print(f"\n✓ Images downloaded to '{path}' folder")
print(f"  - Forest images: {len(list((path/'forest').ls()))}")
print(f"  - Bird images: {len(list((path/'bird').ls()))}")

## Step 4: Visualize Sample Images

Let's see what our training data looks like!

In [ ]:
# Remove any failed downloads
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
print(f"Removed {len(failed)} corrupted images")

# Create a DataBlock and show sample images
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

# Show a batch of training images
dls.show_batch(max_n=9)

## Step 5: Train the Model

Now we'll use **transfer learning** with ResNet18:
- ResNet18 is already trained on millions of images
- We fine-tune it for 3 epochs on our forest/bird data
- This takes just a few minutes!

In [ ]:
# Fix for Colab/Kaggle progress bar issues
import fastprogress.fastprogress as fp
fp.MasterBar, fp.ProgressBar = fp.ConsoleMasterBar, fp.ConsoleProgressBar

# Create a learner using pre-trained ResNet18 (without default progress callbacks)
learn = vision_learner(dls, resnet18, metrics=error_rate, cbs=[])
learn.callback_fns = []  # Remove any callbacks that might add notebook progress

# Fine-tune the model (this trains it on our data)
print("Training the model...")
learn.fine_tune(3)

print("\n✓ Training complete!")

## Step 6: Test the Model

Let's test our trained model on a new bird image!

In [ ]:
# Download a test image
urls = search_images('bird photo', max_images=1)
test_image_path = 'test_bird.jpg'
download_url(urls[0], test_image_path, show_progress=False)

# Make a prediction
img = PILImage.create(test_image_path)
is_bird, _, probs = learn.predict(img)

# Display results
print(f"\n{'='*50}")
print(f"PREDICTION: {is_bird}")
print(f"{'='*50}")
print(f"Probability it's a bird: {probs[1]:.2%}")
print(f"Probability it's a forest: {probs[0]:.2%}")

# Show the test image
img.show()

## Step 7: Try Your Own Image! 🎯

Upload your own image or search for a different one to test.

In [ ]:
# Option 1: Test with a forest image
urls = search_images('forest photo', max_images=1)
download_url(urls[0], 'test_forest.jpg', show_progress=False)

img = PILImage.create('test_forest.jpg')
prediction, _, probs = learn.predict(img)

print(f"Prediction: {prediction}")
print(f"Bird probability: {probs[1]:.2%}")
print(f"Forest probability: {probs[0]:.2%}")
img.show()

## Step 8: Evaluate Model Performance

Let's see how well our model performs on the validation set.

In [ ]:
# Show confusion matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(6,6))

## Step 9: See Top Losses (Most Confused Images)

Which images did the model get most wrong?

In [ ]:
# Show images with highest loss (most uncertain predictions)
interp.plot_top_losses(9, figsize=(15,10))

## Step 10: Save Your Model

Save the trained model so you can use it later!

In [ ]:
# Export the model
learn.export('bird_classifier_model.pkl')
print("✓ Model saved as 'bird_classifier_model.pkl'")

# To load it later, use:
# learn_loaded = load_learner('bird_classifier_model.pkl')

---

## 🎉 Congratulations!

You've successfully:
1. ✅ Downloaded training data automatically
2. ✅ Trained an image classifier using transfer learning
3. ✅ Tested it on new images
4. ✅ Evaluated its performance
5. ✅ Saved your model

### Next Steps:
- Try different categories (cats vs dogs, cars vs trucks, etc.)
- Increase the number of training images for better accuracy
- Try different pre-trained models (resnet34, resnet50)
- Deploy your model as a web app!

### Key Concepts Learned:
- **Transfer Learning**: Using pre-trained models for new tasks
- **Fine-tuning**: Adapting a model to your specific data
- **Data Augmentation**: Automatic image transformations for better training
- **Model Evaluation**: Using confusion matrices and top losses